In [ ]:
import numpy as np
import pathlib
import laspy
import shapely.geometry as sg
import shapely.ops as so
import geopandas as gpd
from tqdm.notebook import tqdm
tqdm.pandas()

from upcp.utils import clip_utils

import set_path
from upc_sw import poly_utils

In [ ]:
ahn_raw_folder = '/Volumes/Daan 1TB/AHN4/Bronbestanden/'
ahn_subtile_folder = '../../datasets/AHN4/AMS_subtiles_1000/'
box_size = 1000

## Data preparation

### Read stadsdelen shapes

In [ ]:
ams_gdf = gpd.read_file('stadsdelen.gpkg')

In [ ]:
ams_gdf.unary_union.bounds

In [ ]:
# Extend bounds to nearest 1000m
ams_bounds = (110000, 476000, 135000, 494000)

### Load pointcloud tiles

In [ ]:
run1_tiles = gpd.read_file('run1_tiles_10k.gpkg')
run2_tiles = gpd.read_file('run2_tiles_10k.gpkg')

In [ ]:
all_tiles = list(set(run1_tiles['tilecode']).union(set(run2_tiles['tilecode'])))
all_tiles.sort()

In [ ]:
run1_tiles = None
run2_tiles = None

In [ ]:
all_tiles_gdf = gpd.GeoDataFrame({'tilecode': all_tiles,
                                  'geometry': [poly_utils.tilecode_to_poly(tc) for tc in all_tiles]})

In [ ]:
all_tiles_merged = gpd.GeoDataFrame({'geometry': [geom for geom in all_tiles_gdf.unary_union.geoms]})

### Create subtiles grid

In [ ]:
def box_to_name(box):
    (x_min, y_min, _, _) = box.bounds
    return f'{x_min/box_size:.0f}_{y_min/box_size:.0f}'

xs = np.arange(ams_bounds[0], ams_bounds[2], box_size)
ys = np.arange(ams_bounds[1], ams_bounds[3], box_size)

geoms = [sg.box(x, y, x+box_size, y+box_size) for x in xs for y in ys]
names = [box_to_name(box) for box in geoms]

ams_subtiles_gdf = gpd.GeoDataFrame({'name': names, 'geometry': geoms})

target_shape = so.unary_union([all_tiles_merged.unary_union, ams_gdf.unary_union])

ams_subtiles_gdf = ams_subtiles_gdf[ams_subtiles_gdf.intersects(target_shape)]

### Load AHN bronbestanden

In [ ]:
ahn_raw_files = list(pathlib.Path(ahn_raw_folder).glob('*.LAZ'))

In [ ]:
bounds = []
for f in ahn_raw_files:
    with laspy.open(f) as las:
        [x_min, y_min] = las.header.mins[0:2]
        [x_max, y_max] = las.header.maxs[0:2]
        bounds.append([x_min, y_min, x_max, y_max])

In [ ]:
ahn_raw_gdf = gpd.GeoDataFrame({'filename': [f.name for f in ahn_raw_files],
                                'geometry': [sg.box(*b) for b in bounds]})

ahn_raw_gdf = ahn_raw_gdf[ahn_raw_gdf.intersects(ams_subtiles_gdf.unary_union)]

### Plotting (optional)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,6))

ahn_raw_gdf.plot(ax=ax, color='grey', alpha=0.25)
ams_subtiles_gdf.plot(ax=ax, color='grey', edgecolor='black', alpha=0.5)
ams_gdf.plot(ax=ax, color='lightblue', alpha=0.5)
all_tiles_merged.plot(ax=ax, color='blue', alpha=0.5)

## Perform splitting

In [ ]:
points_per_iter = int(1e6)
buffer = 5

In [ ]:
ams_subtiles_gdf['geometry'] = ams_subtiles_gdf.buffer(buffer)
ams_subtiles_gdf['bounds'] = ams_subtiles_gdf.apply(lambda row: row['geometry'].bounds, axis=1)

In [ ]:
ams_subtiles_gdf.reset_index(inplace=True)

In [ ]:
pathlib.Path(ahn_subtile_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
ahn_total_points = 0
for f in list(ahn_raw_gdf['filename']):
    file_path = pathlib.Path(ahn_raw_folder) / f
    with laspy.open(file_path) as las:
        ahn_total_points += las.header.point_count

In [ ]:
from typing import List
from typing import Optional

pbar = tqdm(total=ahn_total_points, unit=' points', unit_scale=True, unit_divisor=1000, smoothing=0)

writers: List[Optional[laspy.LasWriter]] = [None] * len(ams_subtiles_gdf)

for _, row in ahn_raw_gdf.iterrows():
    ahn_file = row['filename']
    ahn_shape = row['geometry']
    pbar.set_postfix_str(ahn_file)
    file_path = pathlib.Path(ahn_raw_folder) / ahn_file
    with laspy.open(file_path) as file:
        try:
            for points in file.chunk_iterator(points_per_iter):

                # For performance we need to use copy
                # so that the underlying arrays are contiguous
                points_xy = np.vstack((points.x.copy(), points.y.copy())).T

                for i, row in ams_subtiles_gdf.iterrows():
                    if not sg.box(*row['bounds']).intersects(ahn_shape):
                        continue
                    mask = clip_utils.rectangle_clip(points_xy, row['bounds'])

                    if np.any(mask):
                        if writers[i] is None:
                            output_path = pathlib.Path(ahn_subtile_folder) / f"ahn4_{row['name']}.laz"
                            writers[i] = laspy.open(output_path,
                                                    mode='w',
                                                    header=file.header)
                        sub_points = points[mask]
                        writers[i].write_points(sub_points)

                pbar.update(len(points))
        finally: pass
pbar.close()

for writer in writers:
    if writer is not None:
        writer.close()